In [1]:
import numpy as np 
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

RuntimeError: module was compiled against NumPy C-API version 0x10 (NumPy 1.23) but the running NumPy has C-API version 0xe. Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem.

In [3]:
red = pd.read_csv("C:/Users/HP/Downloads/winequality-red.csv")
white = pd.read_csv("C:/Users/HP/Downloads/winequality-white.csv")

# Red Wine

In [4]:
red.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [6]:
red.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [9]:
from scipy.stats import kruskal

numerical_vars = ["fixed acidity", "volatile acidity","citric acid","residual sugar", "chlorides", "free sulfur dioxide","total sulfur dioxide","density", "pH", "sulphates","alcohol"]
kruskal_results = {}

for var in numerical_vars:
    grouped_data = [red[red["quality"] == level][var] for level in sorted(red["quality"].unique())]
    stat, p = kruskal(*grouped_data)
    kruskal_results[var] = {"Kruskal Statistic": stat, "p-value": p}

kruskal_results

{'fixed acidity': {'Kruskal Statistic': 30.542977739172954,
  'p-value': 1.1528814177914685e-05},
 'volatile acidity': {'Kruskal Statistic': 242.5035332434073,
  'p-value': 2.2298939975568596e-50},
 'citric acid': {'Kruskal Statistic': 93.66965479021577,
  'p-value': 1.1374798403913489e-18},
 'residual sugar': {'Kruskal Statistic': 6.826227461373974,
  'p-value': 0.23388810145429345},
 'chlorides': {'Kruskal Statistic': 72.77669179624459,
  'p-value': 2.7059878583835738e-14},
 'free sulfur dioxide': {'Kruskal Statistic': 31.12837702820869,
  'p-value': 8.83609073000697e-06},
 'total sulfur dioxide': {'Kruskal Statistic': 111.13336495132982,
  'p-value': 2.3602167590663723e-22},
 'density': {'Kruskal Statistic': 65.32854013366436,
  'p-value': 9.579606837067283e-13},
 'pH': {'Kruskal Statistic': 23.736183400155504,
  'p-value': 0.0002439765647383132},
 'sulphates': {'Kruskal Statistic': 232.38659415235918,
  'p-value': 3.2932297464108236e-48},
 'alcohol': {'Kruskal Statistic': 412.37683

In [10]:
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, classification_report

df = red.drop(columns=['residual sugar'])

X = df.drop(columns=['quality'])
y = df['quality']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [11]:
from statsmodels.miscmodels.ordinal_model import OrderedModel

model = OrderedModel(y_train, X_train_scaled, distr='logit')
ordinal_logreg = model.fit()

y_pred_prob = ordinal_logreg.predict(X_test_scaled)
y_pred = y_pred_prob.argmax(axis=1)
conf_matrix = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Test Accuracy: {accuracy:.4f}")
print("Classification Report:\n", report)

Confusion Matrix:
[[ 0  0  0  0  0  0  0]
 [ 1  1  0  0  0  0  0]
 [ 9  2  0  0  0  0  0]
 [99 37  0  0  0  0  0]
 [51 74  3  0  0  0  0]
 [ 5 33  2  0  0  0  0]
 [ 0  3  0  0  0  0  0]]
Test Accuracy: 0.0031
Classification Report:
               precision    recall  f1-score   support

           2       0.00      0.00      0.00         0
           3       0.01      0.50      0.01         2
           4       0.00      0.00      0.00        11
           5       0.00      0.00      0.00       136
           6       0.00      0.00      0.00       128
           7       0.00      0.00      0.00        40
           8       0.00      0.00      0.00         3

    accuracy                           0.00       320
   macro avg       0.00      0.07      0.00       320
weighted avg       0.00      0.00      0.00       320



C:\Users\HP\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)
rf_report = classification_report(y_test, rf_predictions)
conf_matrix_rf = confusion_matrix(y_test, rf_predictions)
accuracy = accuracy_score(y_test, rf_predictions)
single_tree = rf_model.estimators_[0]

print("\n Parameters of the Extracted Decision Tree:")
print(single_tree.get_params())
print(f"Confusion Matrix:\n{conf_matrix_rf}")
print(f"Test Accuracy: {accuracy:.4f}")
print("\nRandom Forest Classification Report:\n", rf_report)


 Parameters of the Extracted Decision Tree:
{'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'random_state': 1608637542, 'splitter': 'best'}
Confusion Matrix:
[[  0   0   1   1   0   0]
 [  0   0   9   2   0   0]
 [  0   0 103  32   1   0]
 [  0   0  28  95   5   0]
 [  0   0   2  17  21   0]
 [  0   0   0   1   1   1]]
Test Accuracy: 0.6875

Random Forest Classification Report:
               precision    recall  f1-score   support

           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00        11
           5       0.72      0.76      0.74       136
           6       0.64      0.74      0.69       128
           7       0.75      0.53      0.62        40
           8       1.00      0.33      0.50         3

    accuracy           

In [13]:
from sklearn.neural_network import MLPClassifier

nn_model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42)
nn_model.fit(X_train_scaled, y_train)

y_pred = nn_model.predict(X_test_scaled)
conf_matrix_nn = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Confusion Matrix:\n{conf_matrix_nn}")
print("MLPClassifier Classification Report:\n", report)

Test Accuracy: 0.6281
Confusion Matrix:
[[  0   1   0   1   0   0]
 [  0   0   9   2   0   0]
 [  0   0 100  34   2   0]
 [  0   0  32  82  13   1]
 [  0   0   3  17  19   1]
 [  0   0   0   2   1   0]]
MLPClassifier Classification Report:
               precision    recall  f1-score   support

           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00        11
           5       0.69      0.74      0.71       136
           6       0.59      0.64      0.62       128
           7       0.54      0.47      0.51        40
           8       0.00      0.00      0.00         3

    accuracy                           0.63       320
   macro avg       0.31      0.31      0.31       320
weighted avg       0.60      0.63      0.61       320



In [14]:
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
knn_model.fit(X_train_scaled, y_train)

y_pred = knn_model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix_knn = confusion_matrix(y_test, y_pred)

print(f"Confusion Matrix:\n{conf_matrix_knn}")
print(f"Test Accuracy: {accuracy:.4f}")
print("Classification Report:\n", report)


Confusion Matrix:
[[ 0  0  0  2  0  0]
 [ 0  0  7  4  0  0]
 [ 0  0 89 44  3  0]
 [ 0  0 40 76 12  0]
 [ 0  0  2 17 21  0]
 [ 0  0  0  2  1  0]]
Test Accuracy: 0.5813
Classification Report:
               precision    recall  f1-score   support

           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00        11
           5       0.64      0.65      0.65       136
           6       0.52      0.59      0.56       128
           7       0.57      0.53      0.55        40
           8       0.00      0.00      0.00         3

    accuracy                           0.58       320
   macro avg       0.29      0.30      0.29       320
weighted avg       0.55      0.58      0.57       320



In [15]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

kernels = ['linear', 'poly', 'sigmoid', 'rbf'] 
for kernel in kernels:
    print(f"Using kernel: {kernel}")
   
    svm = SVC(kernel=kernel, C=1, gamma='scale', decision_function_shape='ovr')
    svm.fit(X_train_scaled, y_train)
   
    y_pred = svm.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)

    print(f"Test Accuracy with {kernel} kernel: {accuracy:.4f}")
    print(f"Classification Report for {kernel} kernel:\n{report}")
    print(f"Confusion Matrix for {kernel} kernel:\n{conf_matrix}\n")


Using kernel: linear
Test Accuracy with linear kernel: 0.5719
Classification Report for linear kernel:
              precision    recall  f1-score   support

           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00        11
           5       0.62      0.77      0.69       136
           6       0.51      0.61      0.56       128
           7       0.00      0.00      0.00        40
           8       0.00      0.00      0.00         3

    accuracy                           0.57       320
   macro avg       0.19      0.23      0.21       320
weighted avg       0.47      0.57      0.52       320

Confusion Matrix for linear kernel:
[[  0   0   1   1   0   0]
 [  0   0  10   1   0   0]
 [  0   0 105  31   0   0]
 [  0   0  50  78   0   0]
 [  0   0   2  38   0   0]
 [  0   0   0   3   0   0]]

Using kernel: poly
Test Accuracy with poly kernel: 0.5938
Classification Report for poly kernel:
              precision    recall  f1-score   support

  

In [ ]:
# 1. Random Forest Feature Importance
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
feature_importance_rf = rf_model.feature_importances_
feature_names = wine_data.drop(columns=['wine_type']).columns

print("Random Forest Feature Importance:")
for name, importance in zip(feature_names, feature_importance_rf):
    print(f"{name}: {importance:.4f}")

# Visualize Random Forest Feature Importance
plt.figure(figsize=(10, 6))
plt.barh(feature_names, feature_importance_rf, color='skyblue')
plt.xlabel('Feature Importance')
plt.ylabel('Features')
plt.title('Random Forest Feature Importance')
plt.show()


# White Wine

In [5]:
white.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [16]:
from scipy.stats import kruskal

numerical_vars = ["fixed acidity", "volatile acidity","citric acid","residual sugar", "chlorides", "free sulfur dioxide","total sulfur dioxide","density", "pH", "sulphates","alcohol"]
kruskal_results = {}

for var in numerical_vars:
    grouped_data = [white[white["quality"] == level][var] for level in sorted(red["quality"].unique())]
    stat, p = kruskal(*grouped_data)
    kruskal_results[var] = {"Kruskal Statistic": stat, "p-value": p}

kruskal_results

{'fixed acidity': {'Kruskal Statistic': 39.018501832207164,
  'p-value': 2.354559482193958e-07},
 'volatile acidity': {'Kruskal Statistic': 285.17002269626676,
  'p-value': 1.542239105986827e-59},
 'citric acid': {'Kruskal Statistic': 11.116480221685407,
  'p-value': 0.04911881317519302},
 'residual sugar': {'Kruskal Statistic': 94.0548955009708,
  'p-value': 9.438590241321224e-19},
 'chlorides': {'Kruskal Statistic': 504.05320642683205,
  'p-value': 1.0650072157363546e-106},
 'free sulfur dioxide': {'Kruskal Statistic': 114.93791814870919,
  'p-value': 3.701241383437549e-23},
 'total sulfur dioxide': {'Kruskal Statistic': 265.02129292831694,
  'p-value': 3.2809847332530804e-55},
 'density': {'Kruskal Statistic': 648.7099427979668,
  'p-value': 6.016269071508691e-138},
 'pH': {'Kruskal Statistic': 60.89567635005582,
  'p-value': 7.93569334747448e-12},
 'sulphates': {'Kruskal Statistic': 13.582043835248522,
  'p-value': 0.018494049526569155},
 'alcohol': {'Kruskal Statistic': 1007.38540

In [17]:
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, classification_report

df = red.drop(columns=['citric acid', 'sulphates'])

X = df.drop(columns=['quality'])
y = df['quality']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [18]:
from statsmodels.miscmodels.ordinal_model import OrderedModel

model = OrderedModel(y_train, X_train_scaled, distr='logit')
ordinal_logreg = model.fit()

y_pred_prob = ordinal_logreg.predict(X_test_scaled)
y_pred = y_pred_prob.argmax(axis=1)
conf_matrix = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Test Accuracy: {accuracy:.4f}")
print("Classification Report:\n", report)

Confusion Matrix:
[[ 0  0  0  0  0  0  0]
 [ 2  0  0  0  0  0  0]
 [ 8  3  0  0  0  0  0]
 [89 46  1  0  0  0  0]
 [50 78  0  0  0  0  0]
 [ 5 35  0  0  0  0  0]
 [ 1  1  1  0  0  0  0]]
Test Accuracy: 0.0000
Classification Report:
               precision    recall  f1-score   support

           2       0.00      0.00      0.00       0.0
           3       0.00      0.00      0.00       2.0
           4       0.00      0.00      0.00      11.0
           5       0.00      0.00      0.00     136.0
           6       0.00      0.00      0.00     128.0
           7       0.00      0.00      0.00      40.0
           8       0.00      0.00      0.00       3.0

    accuracy                           0.00     320.0
   macro avg       0.00      0.00      0.00     320.0
weighted avg       0.00      0.00      0.00     320.0



C:\Users\HP\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)
rf_report = classification_report(y_test, rf_predictions)
conf_matrix_rf = confusion_matrix(y_test, rf_predictions)
accuracy = accuracy_score(y_test, rf_predictions)
single_tree = rf_model.estimators_[0]

print("\n Parameters of the Extracted Decision Tree:")
print(single_tree.get_params())
print(f"Confusion Matrix:\n{conf_matrix_rf}")
print(f"Test Accuracy: {accuracy:.4f}")
print("\nRandom Forest Classification Report:\n", rf_report)


 Parameters of the Extracted Decision Tree:
{'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'random_state': 1608637542, 'splitter': 'best'}
Confusion Matrix:
[[  0   1   1   0   0   0]
 [  0   0   9   2   0   0]
 [  0   0 101  33   2   0]
 [  0   0  32  90   6   0]
 [  0   0   3  17  19   1]
 [  0   0   0   1   1   1]]
Test Accuracy: 0.6594

Random Forest Classification Report:
               precision    recall  f1-score   support

           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00        11
           5       0.69      0.74      0.72       136
           6       0.63      0.70      0.66       128
           7       0.68      0.47      0.56        40
           8       0.50      0.33      0.40         3

    accuracy           

In [20]:
from sklearn.neural_network import MLPClassifier

nn_model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42)
nn_model.fit(X_train_scaled, y_train)

y_pred = nn_model.predict(X_test_scaled)
conf_matrix_nn = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Confusion Matrix:\n{conf_matrix_nn}")
print("MLPClassifier Classification Report:\n", report)

Test Accuracy: 0.5594
Confusion Matrix:
[[ 0  1  1  0  0  0]
 [ 0  0  8  3  0  0]
 [ 0  2 92 39  3  0]
 [ 0  0 36 73 19  0]
 [ 0  0  6 19 14  1]
 [ 0  0  0  1  2  0]]
MLPClassifier Classification Report:
               precision    recall  f1-score   support

           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00        11
           5       0.64      0.68      0.66       136
           6       0.54      0.57      0.56       128
           7       0.37      0.35      0.36        40
           8       0.00      0.00      0.00         3

    accuracy                           0.56       320
   macro avg       0.26      0.27      0.26       320
weighted avg       0.54      0.56      0.55       320



In [21]:
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
knn_model.fit(X_train_scaled, y_train)

y_pred = knn_model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix_knn = confusion_matrix(y_test, y_pred)

print(f"Confusion Matrix:\n{conf_matrix_knn}")
print(f"Test Accuracy: {accuracy:.4f}")
print("Classification Report:\n", report)


Confusion Matrix:
[[ 0  0  1  1  0  0]
 [ 0  0  6  5  0  0]
 [ 0  4 91 39  2  0]
 [ 0  1 44 75  8  0]
 [ 0  0  8 15 17  0]
 [ 0  0  1  0  2  0]]
Test Accuracy: 0.5719
Classification Report:
               precision    recall  f1-score   support

           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00        11
           5       0.60      0.67      0.63       136
           6       0.56      0.59      0.57       128
           7       0.59      0.42      0.49        40
           8       0.00      0.00      0.00         3

    accuracy                           0.57       320
   macro avg       0.29      0.28      0.28       320
weighted avg       0.55      0.57      0.56       320



In [22]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

kernels = ['linear', 'poly', 'sigmoid', 'rbf'] 
for kernel in kernels:
    print(f"Using kernel: {kernel}")
   
    svm = SVC(kernel=kernel, C=1, gamma='scale', decision_function_shape='ovr')
    svm.fit(X_train_scaled, y_train)
   
    y_pred = svm.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)

    print(f"Test Accuracy with {kernel} kernel: {accuracy:.4f}")
    print(f"Classification Report for {kernel} kernel:\n{report}")
    print(f"Confusion Matrix for {kernel} kernel:\n{conf_matrix}\n")


Using kernel: linear
Test Accuracy with linear kernel: 0.5469
Classification Report for linear kernel:
              precision    recall  f1-score   support

           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00        11
           5       0.61      0.72      0.66       136
           6       0.48      0.60      0.54       128
           7       0.00      0.00      0.00        40
           8       0.00      0.00      0.00         3

    accuracy                           0.55       320
   macro avg       0.18      0.22      0.20       320
weighted avg       0.45      0.55      0.50       320

Confusion Matrix for linear kernel:
[[ 0  0  2  0  0  0]
 [ 0  0  9  2  0  0]
 [ 0  0 98 38  0  0]
 [ 0  0 51 77  0  0]
 [ 0  0  1 39  0  0]
 [ 0  0  0  3  0  0]]

Using kernel: poly
Test Accuracy with poly kernel: 0.5750
Classification Report for poly kernel:
              precision    recall  f1-score   support

           3       0.00      0.00     